<a href="https://colab.research.google.com/github/derewor/Crop-Prediction/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sowing Success: How Machine Learning Helps Farmers Select the Best Crops

![Farmer in a field](farmer_in_a_field.jpg)

Measuring essential soil metrics such as nitrogen, phosphorous, potassium levels, and pH value is an important aspect of assessing soil condition. However, it can be an expensive and time-consuming process, which can cause farmers to prioritize which metrics to measure based on their budget constraints.

Farmers have various options when it comes to deciding which crop to plant each season. Their primary objective is to maximize the yield of their crops, taking into account different factors. One crucial factor that affects crop growth is the condition of the soil in the field, which can be assessed by measuring basic elements such as nitrogen and potassium levels. Each crop has an ideal soil condition that ensures optimal growth and maximum yield.

A farmer reached out to you as a machine learning expert for assistance in selecting the best crop for his field. They've provided you with a dataset called `soil_measures.csv`, which contains:

- `"N"`: Nitrogen content ratio in the soil
- `"P"`: Phosphorous content ratio in the soil
- `"K"`: Potassium content ratio in the soil
- `"pH"` value of the soil
- `"crop"`: categorical values that contain various crops (target variable).

Each row in this dataset represents various measures of the soil in a particular field. Based on these measurements, the crop specified in the `"crop"` column is the optimal choice for that field.  

In this project, you will build multi-class classification models to predict the type of `"crop"` and identify the single most importance feature for predictive performance.

In [34]:
# All required libraries are imported here for you.
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# Import Randomforest.
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [3]:
# Load the dataset
crops = pd.read_csv("soil_measures.csv")
df_crops = pd.DataFrame(crops)
# Write your code here
df_crops['crop'].nunique()

22

In [ ]:
# checking for unique crops
df_crops.crop.unique()

array(['rice', 'maize', 'chickpea', 'kidneybeans', 'pigeonpeas',
       'mothbeans', 'mungbean', 'blackgram', 'lentil', 'pomegranate',
       'banana', 'mango', 'grapes', 'watermelon', 'muskmelon', 'apple',
       'orange', 'papaya', 'coconut', 'cotton', 'jute', 'coffee'],
      dtype=object)

## **Data Cleaning**

In [4]:
df_crops.head()

,N,P,K,ph,crop
0,90,42,43,6.502985,rice
1,85,58,41,7.038096,rice
2,60,55,44,7.840207,rice
3,74,35,40,6.980401,rice
4,78,42,42,7.628473,rice


In [ ]:
# checking for nulls
df_crops.isna().sum() # there are not null values

N       0
P       0
K       0
ph      0
crop    0
dtype: int64

In [5]:
# checking for duplicates.
df_crops.duplicated().sum() # There are not dupliated values

np.int64(0)

In [6]:
# Check for the datatypes.
df_crops.dtypes

,0
N,int64
P,int64
K,int64
ph,float64
crop,object


In [7]:
# Describe the fields with the major statstical tools.
df_crops.describe()

,N,P,K,ph
count,2200.000000,2200.000000,2200.000000,2200.000000
mean,50.551818,53.362727,48.149091,6.469480
std,36.917334,32.985883,50.647931,0.773938
min,0.000000,5.000000,5.000000,3.504752
25%,21.000000,28.000000,20.000000,5.971693
50%,37.000000,51.000000,32.000000,6.425045
75%,84.250000,68.000000,49.000000,6.923643
max,140.000000,145.000000,205.000000,9.935091


## **Modeling soil metrics for crops selection**

In [9]:
# spliting the data to train and test samples
col_features = df_crops.drop(columns='crop')
target_feature = df_crops['crop']

In [27]:
# standardize the column feature before spliting.
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(col_features)
X_scaled = pd.DataFrame(X_scaled, columns=col_features.columns)

In [30]:
# examine the sizes of the train and test samples.
X_train, X_test = train_test_split(col_features, test_size=0.20, random_state=42)
y_train, y_test = train_test_split(target_feature, test_size=0.20, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1760, 4)
(440, 4)
(1760,)
(440,)


In [38]:
# Fit the model with the train data for each feature and predict the crop based on the test sample.
feature_performance = {}
for features in ['N','P','K','ph']:
    # model = LogisticRegression()
    model = RandomForestClassifier(n_estimators=100)
    model.fit(X_train[[features]], y_train.values)
    y_pred = model.predict(X_test[[features]])
    f1 = metrics.f1_score(y_test, y_pred, average='weighted')
    feature_performance[features] = f1

print(feature_performance)

{'N': 0.14380466084651905, 'P': 0.20674154939477604, 'K': 0.3213627105606857, 'ph': 0.13933118415602644}


In [39]:
# K produced the best F1 score
# Store in best_predictive_feature dictionary
best_predictive_feature = {"K": feature_performance["K"]}
best_predictive_feature

{'K': 0.3213627105606857}

In [55]:
# fit the model based on the feature combinations
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train[['N', 'P', 'K', 'ph']], y_train)
y_pred = model.predict(X_test[['N', 'P', 'K', 'ph']])
f1 = metrics.f1_score(y_test, y_pred, average='weighted')
print("F1 Score with all features:", f1)

F1 Score with all features: 0.8148046712477377
